In [0]:
import pandas as pd
import numpy as np
import keras
# from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import cross_validate, StratifiedKFold, train_test_split
from sklearn.decomposition import PCA
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import f1_score
# from sklearn.svm import SVC
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
import pickle
import time
import tensorflow as tf
from keras.datasets import cifar10
import cv2




Using TensorFlow backend.


In [0]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [0]:
def f1(y_true, y_pred):
    y_pred = np.rint(y_pred)
    tp = sum(y_true*y_pred)
    tn = sum((1-y_true)*(1-y_pred))
    fp = sum((1-y_true)*y_pred)
    fn = sum(y_true*(1-y_pred))

    p = tp / (tp + fp )
    r = tp / (tp + fn )

    f1 = 2*p*r / (p+r)
    return f1

In [0]:
start_time = time.time()
y = pd.read_csv("/content/gdrive/My Drive/data/y_bush_vs_others.csv", header= None)
y_bush = y.values.ravel()
y = pd.read_csv("/content/gdrive/My Drive/data/y_williams_vs_others.csv", header= None)
y_williams = y.values.ravel()
X = pd.read_csv("/content/gdrive/My Drive/data/X.csv", sep=' ', header= None, dtype= float)
X = X.values
print("--- %s seconds ---" % (time.time() - start_time))

--- 24.997381687164307 seconds ---


In [0]:
X_train, X_test, y_bush_train, y_bush_test = train_test_split(X,y_bush, test_size=1./3, 
                                                                      random_state = 6926, shuffle = True, stratify = y_bush)

In [0]:
X_train = X_train.reshape(8822,64,64,1)
X_test = X_test.reshape(4411,64,64,1)
y_bush_train = y_bush_train.reshape(8822,1)
y_bush_test = y_bush_test.reshape(4411,1)

In [0]:
bush_model = Sequential()
bush_model.add(Conv2D(32, kernel_size=(5, 5), strides=(1, 1), activation='relu',input_shape=(64,64,1)))
bush_model.add(Conv2D(32, kernel_size=(5, 5), strides=(1, 1), activation='relu'))
bush_model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))

bush_model.add(Conv2D(32, kernel_size=(3, 3), strides=(1, 1), activation='relu'))
bush_model.add(Conv2D(32, kernel_size=(3, 3), strides=(1, 1), activation='relu'))
bush_model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))

bush_model.add(Conv2D(32, kernel_size=(3, 3), strides=(1, 1), activation='relu'))
bush_model.add(Conv2D(32, kernel_size=(3, 3), strides=(1, 1), activation='relu'))
bush_model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))

bush_model.add(Flatten())
bush_model.add(Dense(384,activation='relu'))
bush_model.add(Dense(1,activation='sigmoid'))
bush_model.compile(loss=keras.losses.binary_crossentropy, optimizer='adam', metrics=['accuracy'])

In [0]:
(xC_train, yC_train), (xC_test, yC_test) = cifar10.load_data()


In [0]:
# binary labels
for index, y in enumerate(yC_train):
    if not y==1:
        yC_train[index] = 0
        
        
        
for index, y in enumerate(yC_test):
    if not y==1:
        yC_test[index] = 0

In [0]:
xC_test.shape

(10000, 32, 32, 3)

In [0]:
# convert to grayscale 
def rgb2gray(rgb):

    r, g, b = rgb[:,:,0], rgb[:,:,1], rgb[:,:,2]
    gray = 0.2989 * r + 0.5870 * g + 0.1140 * b

    return gray

In [0]:
# and resize to 64*64
xC_train_gray = [None] * 50000
xC_test_gray = [None] * 10000
for index, x in enumerate(xC_train):
  cur = rgb2gray(x)
  xC_train_gray[index] = cv2.resize(cur, dsize=(64,64), interpolation=cv2.INTER_CUBIC)
  
for index, x in enumerate(xC_test):
  cur = rgb2gray(x)
  xC_test_gray[index] = cv2.resize(cur, dsize=(64,64), interpolation=cv2.INTER_CUBIC)

In [0]:
xC_train_gray = np.array(xC_train_gray)
xC_test_gray = np.array(xC_test_gray)

xC_train_gray.shape

(50000, 64, 64)

In [0]:
# reshape it into 32*32*1
xC_train_gray = xC_train_gray.reshape(50000,64,64,1)
xC_test_gray = xC_test_gray.reshape(10000,64,64,1)

In [0]:
xC_test_gray.shape

(10000, 64, 64, 1)

In [0]:
yC_test.shape

(10000, 1)

In [0]:
xC_train_gray.shape

(50000, 64, 64, 1)

In [0]:
yC_train.shape

(50000, 1)

In [0]:
#pre-train the model with xC and yC
csv_logger = keras.callbacks.CSVLogger('/content/gdrive/My Drive/data/bush_training.log')
bush_model.fit(xC_train_gray, yC_train, batch_size = 2000, epochs = 10, verbose =1 ,callbacks=[csv_logger])
print(f1(y_bush_train,bush_model.predict(X_train)))
print(f1(y_bush_test,bush_model.predict(X_test)))

Epoch 1/10
50000/50000 [==============================] - 26s 522us/step - loss: 0.9432 - acc: 0.8664
Epoch 2/10
50000/50000 [==============================] - 20s 397us/step - loss: 0.2749 - acc: 0.9019
Epoch 3/10
50000/50000 [==============================] - 20s 398us/step - loss: 0.2340 - acc: 0.9105
Epoch 4/10
50000/50000 [==============================] - 20s 399us/step - loss: 0.2000 - acc: 0.9217
Epoch 5/10
50000/50000 [==============================] - 20s 399us/step - loss: 0.1827 - acc: 0.9293
Epoch 6/10
50000/50000 [==============================] - 20s 401us/step - loss: 0.1696 - acc: 0.9355
Epoch 7/10
50000/50000 [==============================] - 20s 399us/step - loss: 0.1507 - acc: 0.9434
Epoch 8/10
50000/50000 [==============================] - 20s 399us/step - loss: 0.1493 - acc: 0.9449
Epoch 9/10
50000/50000 [==============================] - 20s 400us/step - loss: 0.1313 - acc: 0.9517
Epoch 10/10
50000/50000 [==============================] - 20s 400us/step - loss: 

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:8: RuntimeWarning: invalid value encountered in true_divide
  


[nan]
[nan]


In [0]:
bush_model.save("/content/gdrive/My Drive/data/pretrain_cifar.h5")

In [0]:
# then train the model on bush
csv_logger = keras.callbacks.CSVLogger('/content/gdrive/My Drive/data/bush_training_after.log')
bush_model.fit(X_train, y_bush_train, batch_size = 500, epochs = 10, verbose =1 ,callbacks=[csv_logger])
print(f1(y_bush_train,bush_model.predict(X_train)))
print(f1(y_bush_test,bush_model.predict(X_test)))

Epoch 1/10
8822/8822 [==============================] - 5s 584us/step - loss: 0.2371 - acc: 0.9600
Epoch 2/10
8822/8822 [==============================] - 3s 355us/step - loss: 0.1712 - acc: 0.9600
Epoch 3/10
8822/8822 [==============================] - 3s 353us/step - loss: 0.1678 - acc: 0.9600
Epoch 4/10
8822/8822 [==============================] - 3s 353us/step - loss: 0.1666 - acc: 0.9600
Epoch 5/10
8822/8822 [==============================] - 3s 352us/step - loss: 0.1631 - acc: 0.9600
Epoch 6/10
8822/8822 [==============================] - 3s 354us/step - loss: 0.1613 - acc: 0.9600
Epoch 7/10
8822/8822 [==============================] - 3s 354us/step - loss: 0.1573 - acc: 0.9600
Epoch 8/10
8822/8822 [==============================] - 3s 354us/step - loss: 0.1493 - acc: 0.9600
Epoch 9/10
8822/8822 [==============================] - 3s 354us/step - loss: 0.1431 - acc: 0.9600
Epoch 10/10
8822/8822 [==============================] - 3s 355us/step - loss: 0.1378 - acc: 0.9600


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:8: RuntimeWarning: invalid value encountered in true_divide
  


[nan]
[nan]


In [0]:
# train again
csv_logger = keras.callbacks.CSVLogger('/content/gdrive/My Drive/data/bush_training_after.log')
bush_model.fit(X_train, y_bush_train, batch_size = 500, epochs = 10, verbose =1 ,callbacks=[csv_logger])
print(f1(y_bush_train,bush_model.predict(X_train)))
print(f1(y_bush_test,bush_model.predict(X_test)))

Epoch 1/10
8822/8822 [==============================] - 3s 354us/step - loss: 0.1352 - acc: 0.9600
Epoch 2/10
8822/8822 [==============================] - 3s 353us/step - loss: 0.1294 - acc: 0.9600
Epoch 3/10
8822/8822 [==============================] - 3s 355us/step - loss: 0.1241 - acc: 0.9600
Epoch 4/10
8822/8822 [==============================] - 3s 354us/step - loss: 0.1211 - acc: 0.9599
Epoch 5/10
8822/8822 [==============================] - 3s 353us/step - loss: 0.1175 - acc: 0.9613
Epoch 6/10
8822/8822 [==============================] - 3s 355us/step - loss: 0.1103 - acc: 0.9626
Epoch 7/10
8822/8822 [==============================] - 3s 354us/step - loss: 0.0997 - acc: 0.9660
Epoch 8/10
8822/8822 [==============================] - 3s 354us/step - loss: 0.0920 - acc: 0.9668
Epoch 9/10
8822/8822 [==============================] - 3s 353us/step - loss: 0.0858 - acc: 0.9691
Epoch 10/10
8822/8822 [==============================] - 3s 355us/step - loss: 0.0790 - acc: 0.9739
[0.246305

In [0]:
# train again
csv_logger = keras.callbacks.CSVLogger('/content/gdrive/My Drive/data/bush_training_after.log')
bush_model.fit(X_train, y_bush_train, batch_size = 500, epochs = 10, verbose =1 ,callbacks=[csv_logger])
print(f1(y_bush_train,bush_model.predict(X_train)))
print(f1(y_bush_test,bush_model.predict(X_test)))

Epoch 1/10
8822/8822 [==============================] - 3s 353us/step - loss: 0.0861 - acc: 0.9714
Epoch 2/10
8822/8822 [==============================] - 3s 354us/step - loss: 0.0734 - acc: 0.9751
Epoch 3/10
8822/8822 [==============================] - 3s 352us/step - loss: 0.0610 - acc: 0.9789
Epoch 4/10
8822/8822 [==============================] - 3s 353us/step - loss: 0.0561 - acc: 0.9818
Epoch 5/10
8822/8822 [==============================] - 3s 354us/step - loss: 0.0509 - acc: 0.9830
Epoch 6/10
8822/8822 [==============================] - 3s 354us/step - loss: 0.0452 - acc: 0.9850
Epoch 7/10
8822/8822 [==============================] - 3s 355us/step - loss: 0.0436 - acc: 0.9855
Epoch 8/10
8822/8822 [==============================] - 3s 354us/step - loss: 0.0369 - acc: 0.9866
Epoch 9/10
8822/8822 [==============================] - 3s 355us/step - loss: 0.0308 - acc: 0.9893
Epoch 10/10
8822/8822 [==============================] - 3s 354us/step - loss: 0.0273 - acc: 0.9904
[0.901325

In [0]:
# train again
csv_logger = keras.callbacks.CSVLogger('/content/gdrive/My Drive/data/bush_training_after.log')
bush_model.fit(X_train, y_bush_train, batch_size = 500, epochs = 10, verbose =1 ,callbacks=[csv_logger])
print(f1(y_bush_train,bush_model.predict(X_train)))
print(f1(y_bush_test,bush_model.predict(X_test)))

Epoch 1/10
8822/8822 [==============================] - 3s 355us/step - loss: 0.0247 - acc: 0.9914
Epoch 2/10
8822/8822 [==============================] - 3s 353us/step - loss: 0.0256 - acc: 0.9908
Epoch 3/10
8822/8822 [==============================] - 3s 354us/step - loss: 0.0187 - acc: 0.9942
Epoch 4/10
8822/8822 [==============================] - 3s 354us/step - loss: 0.0140 - acc: 0.9958
Epoch 5/10
8822/8822 [==============================] - 3s 354us/step - loss: 0.0171 - acc: 0.9941
Epoch 6/10
8822/8822 [==============================] - 3s 355us/step - loss: 0.0175 - acc: 0.9932
Epoch 7/10
8822/8822 [==============================] - 3s 355us/step - loss: 0.0100 - acc: 0.9971
Epoch 8/10
8822/8822 [==============================] - 3s 353us/step - loss: 0.0060 - acc: 0.9983
Epoch 9/10
8822/8822 [==============================] - 3s 353us/step - loss: 0.0042 - acc: 0.9997
Epoch 10/10
8822/8822 [==============================] - 3s 352us/step - loss: 0.0038 - acc: 0.9992
[0.989985

In [0]:
bush_model.save("/content/gdrive/My Drive/data/bush_cifar.model")

In [0]:
# train again
csv_logger = keras.callbacks.CSVLogger('/content/gdrive/My Drive/data/bush_training_after.log')
bush_model.fit(X_train, y_bush_train, batch_size = 500, epochs = 10, verbose =1 ,callbacks=[csv_logger])
print(f1(y_bush_train,bush_model.predict(X_train)))
print(f1(y_bush_test,bush_model.predict(X_test)))

Epoch 1/10
8822/8822 [==============================] - 3s 354us/step - loss: 0.0032 - acc: 0.9992
Epoch 2/10
8822/8822 [==============================] - 3s 353us/step - loss: 0.0033 - acc: 0.9992
Epoch 3/10
8822/8822 [==============================] - 3s 352us/step - loss: 0.0042 - acc: 0.9993
Epoch 4/10
8822/8822 [==============================] - 3s 353us/step - loss: 0.0022 - acc: 0.9997
Epoch 5/10
8822/8822 [==============================] - 3s 354us/step - loss: 0.0012 - acc: 1.0000
Epoch 6/10
8822/8822 [==============================] - 3s 353us/step - loss: 6.6795e-04 - acc: 1.0000
Epoch 7/10
8822/8822 [==============================] - 3s 354us/step - loss: 3.7668e-04 - acc: 1.0000
Epoch 8/10
8822/8822 [==============================] - 3s 355us/step - loss: 2.9262e-04 - acc: 1.0000
Epoch 9/10
8822/8822 [==============================] - 3s 353us/step - loss: 2.4641e-04 - acc: 1.0000
Epoch 10/10
8822/8822 [==============================] - 3s 352us/step - loss: 2.2840e-04 - a

In [0]:
bush_model.save("/content/gdrive/My Drive/data/bush_cifar_0.model")

In [0]:
# train again
csv_logger = keras.callbacks.CSVLogger('/content/gdrive/My Drive/data/bush_training_after.log')
bush_model.fit(X_train, y_bush_train, batch_size = 500, epochs = 10, verbose =1 ,callbacks=[csv_logger])
print(f1(y_bush_train,bush_model.predict(X_train)))
print(f1(y_bush_test,bush_model.predict(X_test)))

Epoch 1/10
8822/8822 [==============================] - 3s 352us/step - loss: 2.0617e-04 - acc: 1.0000
Epoch 2/10
8822/8822 [==============================] - 3s 353us/step - loss: 1.8432e-04 - acc: 1.0000
Epoch 3/10
8822/8822 [==============================] - 3s 353us/step - loss: 1.6158e-04 - acc: 1.0000
Epoch 4/10
8822/8822 [==============================] - 3s 352us/step - loss: 1.4969e-04 - acc: 1.0000
Epoch 5/10
8822/8822 [==============================] - 3s 354us/step - loss: 1.3999e-04 - acc: 1.0000
Epoch 6/10
8822/8822 [==============================] - 3s 351us/step - loss: 1.2809e-04 - acc: 1.0000
Epoch 7/10
8822/8822 [==============================] - 3s 352us/step - loss: 1.1809e-04 - acc: 1.0000
Epoch 8/10
8822/8822 [==============================] - 3s 353us/step - loss: 1.1180e-04 - acc: 1.0000
Epoch 9/10
8822/8822 [==============================] - 3s 352us/step - loss: 1.0427e-04 - acc: 1.0000
Epoch 10/10
8822/8822 [==============================] - 3s 353us/step - 

In [0]:
bush_model.save("/content/gdrive/My Drive/data/bush_cifar_1.model")

In [0]:
# train again
csv_logger = keras.callbacks.CSVLogger('/content/gdrive/My Drive/data/bush_training_after.log')
bush_model.fit(X_train, y_bush_train, batch_size = 500, epochs = 10, verbose =1 ,callbacks=[csv_logger])
print(f1(y_bush_train,bush_model.predict(X_train)))
print(f1(y_bush_test,bush_model.predict(X_test)))

Epoch 1/10
8822/8822 [==============================] - 3s 354us/step - loss: 9.5403e-05 - acc: 1.0000
Epoch 2/10
8822/8822 [==============================] - 3s 352us/step - loss: 9.0360e-05 - acc: 1.0000
Epoch 3/10
8822/8822 [==============================] - 3s 353us/step - loss: 8.3585e-05 - acc: 1.0000
Epoch 4/10
8822/8822 [==============================] - 3s 352us/step - loss: 8.3683e-05 - acc: 1.0000
Epoch 5/10
8822/8822 [==============================] - 3s 351us/step - loss: 7.5836e-05 - acc: 1.0000
Epoch 6/10
8822/8822 [==============================] - 3s 353us/step - loss: 7.1442e-05 - acc: 1.0000
Epoch 7/10
8822/8822 [==============================] - 3s 352us/step - loss: 6.8191e-05 - acc: 1.0000
Epoch 8/10
8822/8822 [==============================] - 3s 351us/step - loss: 6.4245e-05 - acc: 1.0000
Epoch 9/10
8822/8822 [==============================] - 3s 352us/step - loss: 6.1669e-05 - acc: 1.0000
Epoch 10/10
8822/8822 [==============================] - 3s 353us/step - 

In [0]:
bush_model.save("/content/gdrive/My Drive/data/bush_cifar_2.model")

In [0]:
# train again
csv_logger = keras.callbacks.CSVLogger('/content/gdrive/My Drive/data/bush_training_after.log')
bush_model.fit(X_train, y_bush_train, batch_size = 500, epochs = 10, verbose =1 ,callbacks=[csv_logger])
print(f1(y_bush_train,bush_model.predict(X_train)))
print(f1(y_bush_test,bush_model.predict(X_test)))

Epoch 1/10
8822/8822 [==============================] - 3s 354us/step - loss: 5.5944e-05 - acc: 1.0000
Epoch 2/10
8822/8822 [==============================] - 3s 352us/step - loss: 5.4476e-05 - acc: 1.0000
Epoch 3/10
8822/8822 [==============================] - 3s 353us/step - loss: 5.2038e-05 - acc: 1.0000
Epoch 4/10
8822/8822 [==============================] - 3s 353us/step - loss: 4.9804e-05 - acc: 1.0000
Epoch 5/10
8822/8822 [==============================] - 3s 353us/step - loss: 4.7122e-05 - acc: 1.0000
Epoch 6/10
8822/8822 [==============================] - 3s 353us/step - loss: 4.5548e-05 - acc: 1.0000
Epoch 7/10
8822/8822 [==============================] - 3s 353us/step - loss: 4.3740e-05 - acc: 1.0000
Epoch 8/10
8822/8822 [==============================] - 3s 352us/step - loss: 4.2162e-05 - acc: 1.0000
Epoch 9/10
8822/8822 [==============================] - 3s 354us/step - loss: 4.0115e-05 - acc: 1.0000
Epoch 10/10
8822/8822 [==============================] - 3s 354us/step - 

In [0]:
bush_model.save("/content/gdrive/My Drive/data/bush_cifar_3.model")